# ICP算法(Iterative closest point)

ICP是一种经典的点云配准算法，也是SLAM中3D-3D相机位姿估计算法，用来估计两个点云之间的欧式变换$R, t$。在KinFusion中利用RGBD相机进行三维重建用到了ICP算法，但由于深度相机可能位置估计不准，而3D-3D需要利用两次深度信息，导致深度相机使用ICP算法比3D-2D算法误差大，因此在普通相机中一般回避3D-3D的方式。

点云配准分为粗配准(Coarse registration)和精配准(Fine Registration)。粗配准的目的是进行粗糙的配准，为精配准提供较好的变换初值。精配准则是给定一个初始变换，进一步优化得到更精确的变换，ICP属于精配准算法。

## 一、算法介绍
从整体上看，ICP把点云配准问题拆分成了两个子问题
* 找最近点
* 找最优变换

### 1.1 找最近点(Find closest point)
利用初始$R_0, t_0$或上次迭代得到的$R_{k-1}, t_{k-1}$对源点云进行变换得到变换点云，然后将变换点云和目标点云进行比较，首先在目标点云处取点集，找出变换点云中对应目标点云点集的**最近邻点**。

### 1.2 求解最优变换(Find Best Transform)
对于point-to-point的ICP问题(知道源点云和目标点云的对应点坐标)，求解最优变换参数$R, t$具有闭式解的形式，下面进行推导。ICP的损失函数如下所示：

$$Loss(R, t) = \frac{1}{N} \sum_{i=1}^N ||R p_s^i + t - p_t^i||_2^2$$

计算源点云与目标点云的质心：

$$\bar{p_s} = \frac{1}{N}\sum_{i=1}^N p_s^i$$
$$\bar{p_t} = \frac{1}{N}\sum_{i=1}^N p_t^i$$

则求解最优变换即求解如下优化问题

$$\begin{aligned}
\min_{R \in SO(3), t \in \mathscr{R}^3}Loss(R, t) 
&:= \frac{1}{N}\sum_{i=1}^N ||R p_s^i + t - p_t^i + \bar{p_t} - \bar{p_t} - R \bar{p_s} + R \bar{p_s}||_2^2 \\ 
&= \frac{1}{N}\sum_{i=1}^N ||(R (p_s^i - \bar{p_s})  - (p_t^i - \bar{p_t}))+ (t - \bar{p_t}  + R \bar{p_s})||_2^2 \\
&= \frac{1}{N}\sum_{i=1}^N \underbrace{||(R (p_s^i - \bar{p_s})  - (p_t^i - \bar{p_t}))||_2^2}_{(1)} + \underbrace{||(t - \bar{p_t}  + R \bar{p_s})||_2^2}_{(2)} \\
\end{aligned}$$

通过上式可以看到，(2)对优化结果没有影响，可以在优化(1)式得到$R^*$后，令
$t^* = \bar{p_t} - R \bar{p_s}$令(2)为0。因此问题转换为对(1)式关于优化变量$R\in SO(3)$的优化问题。将(1)展开，转化为关于交叉项的优化问题

$$\begin{aligned} 
R^* &= \arg \min_{R \in SO(3)} \sum_{i=1}^N -2(p_t^i - \bar{p_t})R(p_s^i - \bar{p_s}) \\
&= \arg \max_{R \in SO(3)} \sum_{i=1}^N trace(R (p_s^i - \bar{p_s})(p_t^i - \bar{p_t})^{T}) \\
&= \arg \max_{R \in SO(3)} trace(RH) \\
\end{aligned}$$

$$H:= \frac{1}{N}\sum_{i=1}^N(p_s^i - \bar{p_s})(p_t^i - \bar{p_t})^{T}$$

对矩阵H做奇异值分解$H = U\Sigma V^T$，则$R^* = VU^T$，详细的引理证明见[../resources/arun.pdf](../resources/arun.pdf)。

另外需要注意的是$SO(3) = \{RR^T = I|det(R) = 1 \}$假如$VU^T$不满足行列式为1，说明其不是一个旋转变换，而是一个反射(Reflection)。假如点云变换不包括反射变换，那么求解出现这种情况的原因是点云共面(coplanar)或共线(colinear)。

共面会导致有一个旋转矩阵解和一个反射解，而共线会导致有无穷个旋转矩阵和反射解，共面和共线会导致模型的退化。共面、共线对应H的最小特征值为0，因此当$det(R) = -1$时，改变V第三列列向量的符号即可。

综上，ICP的最优变换求解问题变成一个优雅的线性代数问题，整个求解过程可以分为三步:
* 1. 中心化
        $$q_s^i = p_s^i - \bar{p_s}$$
        $$q_t^i = p_t^i - \bar{p_t}$$
* 2. 对中心化点求解R
        $$R^* = \arg \min_{R \in SO(3)}\sum_{i}||q_t^i - Rq_s^i||_2^2$$
   该问题**局部极小点就是全局极小点**，除用上面解析方式求解之外，还可以用李代数表示位姿，目标函数为
       $$\min_{\xi \in \mathscr{R}^3}\sum_{i}||q_t^i - exp(\xi ^{\wedge})q_s^i||_2^2$$

* 3. 求解t
        $$t^* = \bar{p_t} - R^* \bar{p_s}$$

### 1.3 迭代
ICP算法就是不断重复1. 寻找最近邻点 2. 求解最优变换参数 两个步骤，直至收敛。收敛条件可以为$R_k, t_k$变化量小于一阈值，或者loss变化量小于一定值。

## 二、算法优缺点
ICP算法优点：
* 简单，不必对点云进行分割和特征提取
* 初值较好的情况下，可以收敛到较好的结果

ICP算法缺点：
* 找最近邻点开销较大
* 只考虑了点与点之间的距离，当点云密度差距较大时无法收敛到合理的位置

## 三、参考资料
[1] [ICP-知乎](https://zhuanlan.zhihu.com/p/104735380)
